In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import math
from sklearn.feature_selection import RFE
df = pd.read_csv('SecondProvider1patient.csv')
df = df.iloc[:, 2:]
def print_performance(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f'accuracy: {round(acc, 2)}')
    print(f'f1 score: {round(f1, 2)}')
def create_coef_df(vars, lr):
    coef = lr.coef_.tolist()[0]
    df = pd.DataFrame({'variable': vars, 'coefficient': coef})
    df['odd'] = df['coefficient'].apply(math.exp)
    df = df.reindex(df.coefficient.abs().sort_values(ascending = False).index)
    df = df.reset_index(drop=True)
    return (df)
def get_vars(ref, col_name):
    tup_l = list(rfe.support_)
    l = []
    for i in range(len(col_name)):
        if tup_l[i] == True:
            l.append(col_name[i])
    return l
pd.options.display.max_rows = 30

In [11]:
#initial stage 15 vars
features = """ CallDTMonth,CallDTHour,CallDTDayofWeek,IncidentFirstDue_401.0,IncidentFirstDue_402.0,IncidentFirstDue_403.0,IncidentFirstDue_404.0,IncidentFirstDue_405.0,IncidentFirstDue_408.0,IncidentFirstDue_409.0,IncidentFirstDue_410.0,IncidentFirstDue_411.0,IncidentFirstDue_412.0,IncidentFirstDue_413.0,IncidentFirstDue_414.0,IncidentFirstDue_415.0,IncidentFirstDue_416.0,IncidentFirstDue_417.0,IncidentFirstDue_418.0,IncidentFirstDue_419.0,IncidentFirstDue_420.0,IncidentFirstDue_421.0,IncidentFirstDue_422.0,IncidentFirstDue_423.0,IncidentFirstDue_424.0,IncidentFirstDue_425.0,IncidentFirstDue_426.0,IncidentFirstDue_427.0,IncidentFirstDue_428.0,IncidentFirstDue_429.0,IncidentFirstDue_430.0,IncidentFirstDue_431.0,IncidentFirstDue_432.0,IncidentFirstDue_433.0,IncidentFirstDue_434.0,IncidentFirstDue_435.0,IncidentFirstDue_436.0,IncidentFirstDue_437.0,IncidentFirstDue_438.0,IncidentFirstDue_439.0,IncidentFirstDue_440.0,IncidentFirstDue_441.0,IncidentFirstDue_442.0,IncidentFirstDue_444.0,IncidentFirstDue_Outside Fairfax,InitialIncidentType_ACCIBF,InitialIncidentType_ACCIF,InitialIncidentType_ACCIHF,InitialIncidentType_ACCIMF,InitialIncidentType_ACCITF,InitialIncidentType_ACOD,InitialIncidentType_ALRMFD,InitialIncidentType_ALS,InitialIncidentType_AMED,InitialIncidentType_ASLTWF,InitialIncidentType_BLAST,InitialIncidentType_BLS,InitialIncidentType_CAVEIN,InitialIncidentType_CODE1,InitialIncidentType_CPRF,InitialIncidentType_DROWNF,InitialIncidentType_ECOD,InitialIncidentType_ELEV,InitialIncidentType_EMETROF,InitialIncidentType_FACCIF,InitialIncidentType_FBLDG,InitialIncidentType_FDACCF,InitialIncidentType_FDACCIF,InitialIncidentType_FDACCIMF,InitialIncidentType_FDACCITF,InitialIncidentType_FGAPT,InitialIncidentType_FHIRIS,InitialIncidentType_FHOU,InitialIncidentType_FOUT,InitialIncidentType_FTH,InitialIncidentType_FTRKF,InitialIncidentType_FVEHF,InitialIncidentType_GASIN,InitialIncidentType_GASOUT,InitialIncidentType_GFRESCUE,InitialIncidentType_HAZMAT,InitialIncidentType_INVF,InitialIncidentType_LOCKF,InitialIncidentType_ODF,InitialIncidentType_PSERVF,InitialIncidentType_RESCUE,InitialIncidentType_RIVERF,InitialIncidentType_RSWIFT,InitialIncidentType_RTRAP,InitialIncidentType_SHOTF,InitialIncidentType_SIG1F,InitialIncidentType_STABF,InitialIncidentType_STILALRM,InitialIncidentType_SUIAF,InitialIncidentType_SYSIN,InitialIncidentType_WIRES,unit_station_1,unit_station_2,unit_station_4,unit_station_5,unit_station_8,unit_station_9,unit_station_10,unit_station_11,unit_station_12,unit_station_13,unit_station_14,unit_station_15,unit_station_16,unit_station_17,unit_station_18,unit_station_19,unit_station_20,unit_station_21,unit_station_22,unit_station_23,unit_station_24,unit_station_25,unit_station_26,unit_station_27,unit_station_28,unit_station_29,unit_station_30,unit_station_31,unit_station_32,unit_station_34,unit_station_35,unit_station_36,unit_station_37,unit_station_38,unit_station_39,unit_station_40,unit_station_41,unit_station_42,patient_age_1 - 9,patient_age_10 - 19,patient_age_100 - 120,patient_age_20 - 29,patient_age_30 - 39,patient_age_40 - 49,patient_age_50 - 59,patient_age_60 - 69,patient_age_70 - 79,patient_age_80 - 89,patient_age_90 - 99,patient_age_< 1,impression_Injury - Head with L.O.C. (S06.0X9A),impression_GI/GU - Appendicitis Acute Onset (K35.80),impression_Newborn Care - Post-term  (P08.21),impression_CV - Pulmonary Embolism (I26),impression_Infectious - SARS (B97.21),impression_Respiratory - Smoke Inhalation (J70.5),impression_Abuse of Sedative Hypnotic or Anxiolytic (F13),impression_GI/GU - Diarrhea (K59.1),impression_Neuro - Paraplegia (G82.20),impression_Maltreatment - Adult Neglect Suspected (T74.01),impression_Injury - Neck (S19.9),impression_GI/GU - Obesity (E66.9),"impression_Edema, unspecified (R60.9)",impression_Behavioral - Anxiety (F41.9),impression_Injury - Intracranial Injury Not Otherwise Specified (S06.9),impression_EENT - Foreign Body to Ear (T16),impression_Metabolic Disorder - Other (E88.9),"impression_CV - STEMI of other sites, With or Without Chest Pain (I21.2)",impression_Environment - Heatstroke (T67.0),impression_Newborn Care - Respiratory Distress (P22),impression_Newborn Care - Normal (Z38.2),impression_Behavioral - Suicide Attempt (T14.91),impression_Anemia (D64.9),impression_Abuse of Inhalant Substances (huffing) (F15),impression_CV - Cardiac Tamponade (I31.4),impression_OB - Childbirth Uncomplicated (O80),impression_Abuse of Alcohol (ETOH) (F10.9),impression_Abuse of Psychoactive Substance (F19),impression_Mobility - Bedridden (Z74.01),impression_Respiratory - Asthma Exacerbation (J45.901),impression_Neuro - Quadriplegia (G82.50),impression_Respiratory - Distress Due to Chemicals Gases Fumes or Vapors (J68.9),impression_Respiratory - Acute Distress/ Breathing Difficulty (J80),impression_Pain - Not Elsewhere Mentioned Sudden Onset (G89.1),impression_Injury - Epidural Hemorrhage from Trauma (S06.4),impression_OB - Preterm Labor with Delivery (O60.1),impression_Respiratory - Bronchospasm Acute Onset (J98.01),impression_Neuro - TIA (transient ischemic attack) (G45.9),impression_Injury - Lung Hemothorax - Traumatic (S27.1),impression_OB - Spontaneous Rupture of Membranes (SROM) (O42.02),impression_GI/GU - Nausea (With Vomiting) (R11.2),impression_GI/GU - Foreign Body Genitourinary Tract (T19.9),impression_GI/GU - GI Problem Not Otherwise Listed (K92.9),impression_Respiratory - Foreign Body Airway (T17.9),impression_Cancer (D49),impression_Newborn Care - Care Not otherwise Listed (P15),impression_Infectious - Encephalitis or Encephalomyelitis (G04.90),impression_Malaise (R53.81),impression_Respiratory - Arrest/Apnea (R06.81),impression_Lab - Hyperkalemia (E87.5),impression_Respiratory - Hyperventialtion (R06.4),impression_Endocrine Disorder - Otherwise Not Listed (E34.9),impression_Infectious - Influenza (Flu Like Symptoms) (J11),impression_Fever (R50.9),impression_Syncope - Syncopal/Fainting Episode (or Near) (R55),impression_Injury - Not Otherwise Listed (T14.90),impression_Injury - Wrist Hand or Fingers (S69.9),impression_EENT - Eye Pain (Non-traumatic) (H57.10),impression_Dental/Tooth Pain (K08.8),impression_Injury - Elbow (S59.90),impression_CV - Hypertension (I10),impression_Abuse of Cocaine (F14),impression_Environment - Hypothermia (T68),impression_Not Applicable,impression_OB - Spontaneous Abortion (Miscarriage) (O03),impression_Respiratory - Pulmonary Edema Acute Onset (J81.0),impression_Burn - Unspecified Degree (T30.0),impression_Environment - Frostbite With Tissue Necrosis (T34.90),impression_Infectious - Epiglottitis (J05.1),impression_Injury - Head without L.O.C. (S06.0X0A),impression_Respiratory - COPD Exacerbation (J44.1),impression_Abuse of Alcohol - Intoxication (F10.92),impression_Injury - Subarachnoid Hemorrhage from Trauma (S06.6),impression_Abuse of Alcohol (ETOH) - Intoxication (F10.92),impression_Respiratory - Not Otherwise Listed (J98.9),impression_Injury - Ankle (S99.91),impression_Newborn Care - Pre-Term (P07.3),impression_Endocrine - Hyperglycemia - Diabetic (E13.65),impression_Behavioral - Hallucination - Visual (R44.1),impression_Injury - Forearm (S59.91),impression_Abuse of Alcohol (ETOH) - Withdrawal (F10.239),impression_CV - Abdominal Aortic Aneurysm (I71.9),impression_Environment - Toxic Exposure (Accidental) (T65.91),impression_GI Bleed - Hematemesis (Bloody Vomit) (K92.0),impression_Dehydration (E86.0),impression_Environment - Suspected Exposure to a Health Hazard (Z77.9),impression_Respiratory - COPD (Emphysema/Chronic Bronchitis) Exacerbation (J44.1),impression_Burn - Third degree (L55.2),impression_Neuro - Stroke/CVA (I63.9),impression_Environment - Heat Exhaustion/Stroke (T67.5),impression_GI Bleed - Melena (Bloody Stool) (K92.1),impression_CV - Congestive Heart Failure (CHF) (I50.9),impression_OB- OB/GYN Complaint Not Otherwise Listed (O26.90),impression_Injury - Hip (S79.91),impression_Inhalant related disorders (F18),impression_Behavioral - Disorientation (R41.0),impression_Injury - Lower Back (S39.92),impression_GI/GU - Vaginal Bleeding (N93.9),impression_Environment - Poisonous Snake Bite (T63.0),impression_Respiratory - Acute Onset Distress (J80),impression_Maltreatment - Child Neglect Suspected (T74.02),"impression_Respiratory - Distress Due to Chemicals, Gases, Fumes, or Vapors (J68.9)",impression_OB - Childbirth Complicated  (O75),impression_Maltreatment - Adult Sexual Abuse/Rape Suspected (T74.21),impression_Behavioral - Mental Disorder Not Otherwise Listed (F99),impression_Infectious - Pneumonia (J18.9),impression_Infectious - Bronchitis - Acute (J20.9),impression_Injury - Abdomen (S39.91),impression_Neuro - Headache - Migraine (G43.9),impression_Environment - Effects of Air or Water Pressure (T70),impression_Injury - Lung Pneumothorax - Traumatic (S27.0),impression_CV - STEMI of other sitesWith or Without Chest Pain (I21.2),impression_EENT - Epistaxis (Non-traumatic) (R04.0),impression_Burn - Second degree (L55.1),impression_Injury - Shoulder or Upper Arm (S49.9),impression_Maltreatment - Adult Physical Abuse Suspected (T74.11),impression_Respiratory - Asphyxia/Suffocation (T71.9),impression_Respiratory - Hemoptysis (R04.2),impression_GI/GU - Foreign  Body Digestive System (T18.9),impression_Abuse of Alcohol (F10.9),impression_OB - Postpartum Hemorrhage Immediate (O72.1),impression_Environment - Toxic Exposure (Intentional) (T65.92),impression_Vertigo (R42),impression_Injury - Genitalia (S39.94),impression_Respiratory - Foreign Body / Airway Obstruction (T17.9),"impression_CV - Myocardial Infarction (Non-STEMI), With or Without Chest Pain (I21.4)",impression_Injury - Pelvis (S39.93),impression_Infectious - Sepsis (A41.9),impression_OB - Postpartum Hemorrhage (O72),impression_Pain - Extremity (Non-traumatic) (M79.609),impression_Abuse of Alcohol - Withdrawal (F10.239),impression_Neuro - Headache (R51),impression_Environment - Heat Exhaustion (T67.5),impression_Neuro - Unconscious (R40.20),"impression_Abuse of Sedative, Hypnotic or Anxiolytic (F13)",impression_Environment - Electrocution (T75.4),impression_CV - Myocardial Infarction (Non-STEMI)With or Without Chest Pain (I21.4),impression_nan,impression_Environment - Stings/Venomous Bites (T63.4),impression_GI/GU - Abdominal Pain Acute Onset  (R10.0),impression_Behavioral - Suicidal/Homicidal Ideation (R45.85),impression_OB - Retained Placenta without Hemorrhage (O73.0),impression_Environment - Poisoning/Drug Ingestion (T65.9),impression_Injury - Ear (S09.91),impression_Environment - Frostbite Superficial (T33.90),impression_CV - STEMI of Anterior WallWith or Without Chest Pain (I21.0),"impression_Encounter for general examination without complaint, suspected or reported (Z00)",impression_Neuro - Neuro Problem Not Otherwise Listed (G99.8),impression_CV - Chest Pain - Presumed Cardiac (I20.9),impression_Pain - Back (Non-traumatic) (M54.9),impression_CV - Hypotension (I95.9),impression_Congenital Deformity (Q89.9),impression_Anaphylactic Shock (T78.2),impression_OB - Pre-eclampsia (O14.9),impression_Behavioral - Depression (F32.9),impression_Injury - Thigh (upper leg) (S79.92),impression_CV - Cardiac Arrhythmia/Dysrhythmia (I49.9),impression_Behavioral - Hallucination - Auditory (R44.0),impression_Behavioral - Hostile (R45.5),impression_Neuro - Seizure (G40.909),impression_Bleeding or Hematoma Post Procedure/Medical Device  (L76.22),impression_Weakness (R53.1),impression_Infectious - Meningitis (G03.9),impression_Syncope - Syncopal Episode (or Near) (R55),impression_Abuse of Narcotic / Opioid / Heroin (Overdose) (F11),impression_Neuro - Hemiplegia (G81.90),impression_Abuse of Hallucinogen (F16),impression_Encounter for general examination without complaintsuspected or reported (Z00),impression_GI/GU - GU Problem Not Otherwise Listed (N39.9),impression_Endocrine Hypoglycemia - Non-diabetic (E16.2),impression_Environment - Lightning Strike (T75.0),impression_GI/GU - Esophageal Obstruction (K22.2),impression_Injury - Thorax (upper chest) (S29.9),impression_Infectious - Disease Unspecified (B99.9),impression_Pain - Chronic (G89.2),impression_Neuro - Altered Mental Status / Level of Consciousness (R41.82),impression_Burn - First degree (L55.0),impression_GI/GU - GERD (Reflux) (K21),impression_Injury - Face (S09.93),impression_Maltreatment - Child Sexual Abuse/Rape Suspected (T74.22),impression_Infectious - RSV (B97.4),impression_GI/GU - Constipation (K59.00),impression_Pain - Chest (presumed non-cardiac) (R07.89),impression_Injury - Eye and/or Orbit (S05),impression_Bleeding or Hematoma from Procedure/Medical Device (L76.22),impression_Sickle Cell Anemia/Crisis (D57.0),impression_EENT - Foreign Body to Eye (T15),impression_Newborn Care - Meconium Aspiration (P24.0),impression_Behavioral - Violent (R45.6),"impression_Injury - Wrist, Hand, or Fingers (S69.9)",impression_Maltreatment - Child Physical Abuse Suspected (T74.12),impression_Pain - Neck (Non-traumatic) (M54.2),impression_Injury - Foot (S99.92),impression_GI/GU - Abdominal Generalized (R10.84),impression_Respiratory - Hyperventilation (R06.4),impression_Neuro - Visual Disturbance (H53.9),impression_Endocrine - Hypoglycemia - Diabetic (E13.64),impression_GI/GU - Abdominal Pain/Problem (R10.84),impression_Infectious - Common Cold (J00),impression_CV - Cardiac Arrest (I46.9),impression_GI/GU - Pelvic or Perineal Pain (R10.2),impression_Mobility - Reduced (Z74.09),impression_Behavioral - Strange Behavior (R46.2),impression_CV - Hypovolemic Shock (T79.4XXA),impression_Environment - Altitude Sickness (T70.29),impression_Allergic Reaction (T78.40),impression_Injury - Nose (S09.92),impression_Respiratory - Pneumothorax (Spontaneous) (J93.9),"impression_CV - STEMI of Anterior Wall, With or Without Chest Pain (I21.0)",impression_Environment - Decompression Sickness (e.g. scuba diving) (T70.3),impression_OB - Contractions (O62.0),impression_Neuro - Status Epilepticus (G40.901),impression_Injury - Subdural Hemorrhage from Trauma (S06.5),impression_OB - Obstetric Trauma (O71.9),"impression_CV - STEMI of Inferior Wall, With or Without Chest Pain (I21.1)",impression_CV - Cardiac Arrest/Obvious Death (R99),impression_CV - Chest Pain - Angina (I20.0),impression_Paralysis - Quadriplegia (G82.50),impression_GI/GU - Hematemesis (vomiting blood) (K92.0),impression_GI/GU - Melena (bloody stools) (K92.1),impression_OB - Preterm Labor Without Delivery (O60.0),impression_Neuro - Altered Mental Status (R41.82),impression_Infectious - Croup (J05.0),impression_Injury - Lower leg (S89.9),impression_GI/GU - Bowel Obstruction (K56.60),impression_OB - Vomiting Due to Pregnancy (O21.9),impression_Endocrine - Adrenocortical Insufficiency (E27.40),impression_CV - STEMI of Inferior WallWith or Without Chest Pain (I21.1),impression_GI/GU - Nausea (Without Vomiting) (R11.0),impression_Intracranial - Stroke (CVA) Hemorrhagic (I62.9)"""
features = features.split(',')
cols = list(df.columns)
fea = [x for x in cols if x in features]
x = df[fea]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=lr, n_features_to_select=15)
rfe.fit(x_train, y_train)
#print_performance(y_test, rfe.predict(x_test))
# l = ['InitialIncidentType_SHOTF',
#  'InitialIncidentType_STABF',
#  'impression_Injury - Intracranial Injury Not Otherwise Specified (S06.9)',
#  'impression_Injury - Lung Hemothorax - Traumatic (S27.1)',
#  'impression_Respiratory - Arrest/Apnea (R06.81)',
#  'impression_Respiratory - Pulmonary Edema Acute Onset (J81.0)',
#  'impression_Neuro - Headache - Migraine (G43.9)',
#  'impression_CV - STEMI of other sitesWith or Without Chest Pain (I21.2)',
#  'impression_Vertigo (R42)',
#  'impression_Neuro - Unconscious (R40.20)',
#  'impression_CV - STEMI of Anterior WallWith or Without Chest Pain (I21.0)',
#  'impression_Behavioral - Violent (R45.6)',
#  'impression_CV - Cardiac Arrest (I46.9)',
#  'impression_CV - Cardiac Arrest/Obvious Death (R99)',
#  'impression_Intracranial - Stroke (CVA) Hemorrhagic (I62.9)']
l = get_vars(rfe, fea)
lr_ini15 = LogisticRegression(max_iter=1000)
x = df[l]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr_ini15.fit(x_train, y_train)
#print_performance(y_test, lr_15.predict(x_test))
ini15_df = create_coef_df(l, lr_ini15)
print(ini15_df)
print_performance(y_test, lr_ini15.predict(x_test))

                                             variable  coefficient        odd
0              impression_CV - Cardiac Arrest (I46.9)     4.056086  57.747842
1                           InitialIncidentType_SHOTF     2.635339  13.948039
2   impression_Respiratory - Pulmonary Edema Acute...     2.365118  10.645293
3             impression_Behavioral - Violent (R45.6)     2.354358  10.531363
4   impression_CV - STEMI of Anterior WallWith or ...     2.278123   9.758344
5                           InitialIncidentType_STABF     2.040926   7.697737
6   impression_CV - Cardiac Arrest/Obvious Death (...     2.011028   7.470996
7   impression_CV - STEMI of other sitesWith or Wi...     1.996092   7.360234
8   impression_Intracranial - Stroke (CVA) Hemorrh...     1.986553   7.290361
9             impression_Neuro - Unconscious (R40.20)     1.957532   7.081831
10  impression_Injury - Lung Hemothorax - Traumati...     1.745040   5.726128
11  impression_Injury - Intracranial Injury Not Ot...     1.7092

In [13]:
#initial stage 25 vars
features = """ CallDTMonth,CallDTHour,CallDTDayofWeek,IncidentFirstDue_401.0,IncidentFirstDue_402.0,IncidentFirstDue_403.0,IncidentFirstDue_404.0,IncidentFirstDue_405.0,IncidentFirstDue_408.0,IncidentFirstDue_409.0,IncidentFirstDue_410.0,IncidentFirstDue_411.0,IncidentFirstDue_412.0,IncidentFirstDue_413.0,IncidentFirstDue_414.0,IncidentFirstDue_415.0,IncidentFirstDue_416.0,IncidentFirstDue_417.0,IncidentFirstDue_418.0,IncidentFirstDue_419.0,IncidentFirstDue_420.0,IncidentFirstDue_421.0,IncidentFirstDue_422.0,IncidentFirstDue_423.0,IncidentFirstDue_424.0,IncidentFirstDue_425.0,IncidentFirstDue_426.0,IncidentFirstDue_427.0,IncidentFirstDue_428.0,IncidentFirstDue_429.0,IncidentFirstDue_430.0,IncidentFirstDue_431.0,IncidentFirstDue_432.0,IncidentFirstDue_433.0,IncidentFirstDue_434.0,IncidentFirstDue_435.0,IncidentFirstDue_436.0,IncidentFirstDue_437.0,IncidentFirstDue_438.0,IncidentFirstDue_439.0,IncidentFirstDue_440.0,IncidentFirstDue_441.0,IncidentFirstDue_442.0,IncidentFirstDue_444.0,IncidentFirstDue_Outside Fairfax,InitialIncidentType_ACCIBF,InitialIncidentType_ACCIF,InitialIncidentType_ACCIHF,InitialIncidentType_ACCIMF,InitialIncidentType_ACCITF,InitialIncidentType_ACOD,InitialIncidentType_ALRMFD,InitialIncidentType_ALS,InitialIncidentType_AMED,InitialIncidentType_ASLTWF,InitialIncidentType_BLAST,InitialIncidentType_BLS,InitialIncidentType_CAVEIN,InitialIncidentType_CODE1,InitialIncidentType_CPRF,InitialIncidentType_DROWNF,InitialIncidentType_ECOD,InitialIncidentType_ELEV,InitialIncidentType_EMETROF,InitialIncidentType_FACCIF,InitialIncidentType_FBLDG,InitialIncidentType_FDACCF,InitialIncidentType_FDACCIF,InitialIncidentType_FDACCIMF,InitialIncidentType_FDACCITF,InitialIncidentType_FGAPT,InitialIncidentType_FHIRIS,InitialIncidentType_FHOU,InitialIncidentType_FOUT,InitialIncidentType_FTH,InitialIncidentType_FTRKF,InitialIncidentType_FVEHF,InitialIncidentType_GASIN,InitialIncidentType_GASOUT,InitialIncidentType_GFRESCUE,InitialIncidentType_HAZMAT,InitialIncidentType_INVF,InitialIncidentType_LOCKF,InitialIncidentType_ODF,InitialIncidentType_PSERVF,InitialIncidentType_RESCUE,InitialIncidentType_RIVERF,InitialIncidentType_RSWIFT,InitialIncidentType_RTRAP,InitialIncidentType_SHOTF,InitialIncidentType_SIG1F,InitialIncidentType_STABF,InitialIncidentType_STILALRM,InitialIncidentType_SUIAF,InitialIncidentType_SYSIN,InitialIncidentType_WIRES,unit_station_1,unit_station_2,unit_station_4,unit_station_5,unit_station_8,unit_station_9,unit_station_10,unit_station_11,unit_station_12,unit_station_13,unit_station_14,unit_station_15,unit_station_16,unit_station_17,unit_station_18,unit_station_19,unit_station_20,unit_station_21,unit_station_22,unit_station_23,unit_station_24,unit_station_25,unit_station_26,unit_station_27,unit_station_28,unit_station_29,unit_station_30,unit_station_31,unit_station_32,unit_station_34,unit_station_35,unit_station_36,unit_station_37,unit_station_38,unit_station_39,unit_station_40,unit_station_41,unit_station_42,patient_age_1 - 9,patient_age_10 - 19,patient_age_100 - 120,patient_age_20 - 29,patient_age_30 - 39,patient_age_40 - 49,patient_age_50 - 59,patient_age_60 - 69,patient_age_70 - 79,patient_age_80 - 89,patient_age_90 - 99,patient_age_< 1,impression_Injury - Head with L.O.C. (S06.0X9A),impression_GI/GU - Appendicitis Acute Onset (K35.80),impression_Newborn Care - Post-term  (P08.21),impression_CV - Pulmonary Embolism (I26),impression_Infectious - SARS (B97.21),impression_Respiratory - Smoke Inhalation (J70.5),impression_Abuse of Sedative Hypnotic or Anxiolytic (F13),impression_GI/GU - Diarrhea (K59.1),impression_Neuro - Paraplegia (G82.20),impression_Maltreatment - Adult Neglect Suspected (T74.01),impression_Injury - Neck (S19.9),impression_GI/GU - Obesity (E66.9),"impression_Edema, unspecified (R60.9)",impression_Behavioral - Anxiety (F41.9),impression_Injury - Intracranial Injury Not Otherwise Specified (S06.9),impression_EENT - Foreign Body to Ear (T16),impression_Metabolic Disorder - Other (E88.9),"impression_CV - STEMI of other sites, With or Without Chest Pain (I21.2)",impression_Environment - Heatstroke (T67.0),impression_Newborn Care - Respiratory Distress (P22),impression_Newborn Care - Normal (Z38.2),impression_Behavioral - Suicide Attempt (T14.91),impression_Anemia (D64.9),impression_Abuse of Inhalant Substances (huffing) (F15),impression_CV - Cardiac Tamponade (I31.4),impression_OB - Childbirth Uncomplicated (O80),impression_Abuse of Alcohol (ETOH) (F10.9),impression_Abuse of Psychoactive Substance (F19),impression_Mobility - Bedridden (Z74.01),impression_Respiratory - Asthma Exacerbation (J45.901),impression_Neuro - Quadriplegia (G82.50),impression_Respiratory - Distress Due to Chemicals Gases Fumes or Vapors (J68.9),impression_Respiratory - Acute Distress/ Breathing Difficulty (J80),impression_Pain - Not Elsewhere Mentioned Sudden Onset (G89.1),impression_Injury - Epidural Hemorrhage from Trauma (S06.4),impression_OB - Preterm Labor with Delivery (O60.1),impression_Respiratory - Bronchospasm Acute Onset (J98.01),impression_Neuro - TIA (transient ischemic attack) (G45.9),impression_Injury - Lung Hemothorax - Traumatic (S27.1),impression_OB - Spontaneous Rupture of Membranes (SROM) (O42.02),impression_GI/GU - Nausea (With Vomiting) (R11.2),impression_GI/GU - Foreign Body Genitourinary Tract (T19.9),impression_GI/GU - GI Problem Not Otherwise Listed (K92.9),impression_Respiratory - Foreign Body Airway (T17.9),impression_Cancer (D49),impression_Newborn Care - Care Not otherwise Listed (P15),impression_Infectious - Encephalitis or Encephalomyelitis (G04.90),impression_Malaise (R53.81),impression_Respiratory - Arrest/Apnea (R06.81),impression_Lab - Hyperkalemia (E87.5),impression_Respiratory - Hyperventialtion (R06.4),impression_Endocrine Disorder - Otherwise Not Listed (E34.9),impression_Infectious - Influenza (Flu Like Symptoms) (J11),impression_Fever (R50.9),impression_Syncope - Syncopal/Fainting Episode (or Near) (R55),impression_Injury - Not Otherwise Listed (T14.90),impression_Injury - Wrist Hand or Fingers (S69.9),impression_EENT - Eye Pain (Non-traumatic) (H57.10),impression_Dental/Tooth Pain (K08.8),impression_Injury - Elbow (S59.90),impression_CV - Hypertension (I10),impression_Abuse of Cocaine (F14),impression_Environment - Hypothermia (T68),impression_Not Applicable,impression_OB - Spontaneous Abortion (Miscarriage) (O03),impression_Respiratory - Pulmonary Edema Acute Onset (J81.0),impression_Burn - Unspecified Degree (T30.0),impression_Environment - Frostbite With Tissue Necrosis (T34.90),impression_Infectious - Epiglottitis (J05.1),impression_Injury - Head without L.O.C. (S06.0X0A),impression_Respiratory - COPD Exacerbation (J44.1),impression_Abuse of Alcohol - Intoxication (F10.92),impression_Injury - Subarachnoid Hemorrhage from Trauma (S06.6),impression_Abuse of Alcohol (ETOH) - Intoxication (F10.92),impression_Respiratory - Not Otherwise Listed (J98.9),impression_Injury - Ankle (S99.91),impression_Newborn Care - Pre-Term (P07.3),impression_Endocrine - Hyperglycemia - Diabetic (E13.65),impression_Behavioral - Hallucination - Visual (R44.1),impression_Injury - Forearm (S59.91),impression_Abuse of Alcohol (ETOH) - Withdrawal (F10.239),impression_CV - Abdominal Aortic Aneurysm (I71.9),impression_Environment - Toxic Exposure (Accidental) (T65.91),impression_GI Bleed - Hematemesis (Bloody Vomit) (K92.0),impression_Dehydration (E86.0),impression_Environment - Suspected Exposure to a Health Hazard (Z77.9),impression_Respiratory - COPD (Emphysema/Chronic Bronchitis) Exacerbation (J44.1),impression_Burn - Third degree (L55.2),impression_Neuro - Stroke/CVA (I63.9),impression_Environment - Heat Exhaustion/Stroke (T67.5),impression_GI Bleed - Melena (Bloody Stool) (K92.1),impression_CV - Congestive Heart Failure (CHF) (I50.9),impression_OB- OB/GYN Complaint Not Otherwise Listed (O26.90),impression_Injury - Hip (S79.91),impression_Inhalant related disorders (F18),impression_Behavioral - Disorientation (R41.0),impression_Injury - Lower Back (S39.92),impression_GI/GU - Vaginal Bleeding (N93.9),impression_Environment - Poisonous Snake Bite (T63.0),impression_Respiratory - Acute Onset Distress (J80),impression_Maltreatment - Child Neglect Suspected (T74.02),"impression_Respiratory - Distress Due to Chemicals, Gases, Fumes, or Vapors (J68.9)",impression_OB - Childbirth Complicated  (O75),impression_Maltreatment - Adult Sexual Abuse/Rape Suspected (T74.21),impression_Behavioral - Mental Disorder Not Otherwise Listed (F99),impression_Infectious - Pneumonia (J18.9),impression_Infectious - Bronchitis - Acute (J20.9),impression_Injury - Abdomen (S39.91),impression_Neuro - Headache - Migraine (G43.9),impression_Environment - Effects of Air or Water Pressure (T70),impression_Injury - Lung Pneumothorax - Traumatic (S27.0),impression_CV - STEMI of other sitesWith or Without Chest Pain (I21.2),impression_EENT - Epistaxis (Non-traumatic) (R04.0),impression_Burn - Second degree (L55.1),impression_Injury - Shoulder or Upper Arm (S49.9),impression_Maltreatment - Adult Physical Abuse Suspected (T74.11),impression_Respiratory - Asphyxia/Suffocation (T71.9),impression_Respiratory - Hemoptysis (R04.2),impression_GI/GU - Foreign  Body Digestive System (T18.9),impression_Abuse of Alcohol (F10.9),impression_OB - Postpartum Hemorrhage Immediate (O72.1),impression_Environment - Toxic Exposure (Intentional) (T65.92),impression_Vertigo (R42),impression_Injury - Genitalia (S39.94),impression_Respiratory - Foreign Body / Airway Obstruction (T17.9),"impression_CV - Myocardial Infarction (Non-STEMI), With or Without Chest Pain (I21.4)",impression_Injury - Pelvis (S39.93),impression_Infectious - Sepsis (A41.9),impression_OB - Postpartum Hemorrhage (O72),impression_Pain - Extremity (Non-traumatic) (M79.609),impression_Abuse of Alcohol - Withdrawal (F10.239),impression_Neuro - Headache (R51),impression_Environment - Heat Exhaustion (T67.5),impression_Neuro - Unconscious (R40.20),"impression_Abuse of Sedative, Hypnotic or Anxiolytic (F13)",impression_Environment - Electrocution (T75.4),impression_CV - Myocardial Infarction (Non-STEMI)With or Without Chest Pain (I21.4),impression_nan,impression_Environment - Stings/Venomous Bites (T63.4),impression_GI/GU - Abdominal Pain Acute Onset  (R10.0),impression_Behavioral - Suicidal/Homicidal Ideation (R45.85),impression_OB - Retained Placenta without Hemorrhage (O73.0),impression_Environment - Poisoning/Drug Ingestion (T65.9),impression_Injury - Ear (S09.91),impression_Environment - Frostbite Superficial (T33.90),impression_CV - STEMI of Anterior WallWith or Without Chest Pain (I21.0),"impression_Encounter for general examination without complaint, suspected or reported (Z00)",impression_Neuro - Neuro Problem Not Otherwise Listed (G99.8),impression_CV - Chest Pain - Presumed Cardiac (I20.9),impression_Pain - Back (Non-traumatic) (M54.9),impression_CV - Hypotension (I95.9),impression_Congenital Deformity (Q89.9),impression_Anaphylactic Shock (T78.2),impression_OB - Pre-eclampsia (O14.9),impression_Behavioral - Depression (F32.9),impression_Injury - Thigh (upper leg) (S79.92),impression_CV - Cardiac Arrhythmia/Dysrhythmia (I49.9),impression_Behavioral - Hallucination - Auditory (R44.0),impression_Behavioral - Hostile (R45.5),impression_Neuro - Seizure (G40.909),impression_Bleeding or Hematoma Post Procedure/Medical Device  (L76.22),impression_Weakness (R53.1),impression_Infectious - Meningitis (G03.9),impression_Syncope - Syncopal Episode (or Near) (R55),impression_Abuse of Narcotic / Opioid / Heroin (Overdose) (F11),impression_Neuro - Hemiplegia (G81.90),impression_Abuse of Hallucinogen (F16),impression_Encounter for general examination without complaintsuspected or reported (Z00),impression_GI/GU - GU Problem Not Otherwise Listed (N39.9),impression_Endocrine Hypoglycemia - Non-diabetic (E16.2),impression_Environment - Lightning Strike (T75.0),impression_GI/GU - Esophageal Obstruction (K22.2),impression_Injury - Thorax (upper chest) (S29.9),impression_Infectious - Disease Unspecified (B99.9),impression_Pain - Chronic (G89.2),impression_Neuro - Altered Mental Status / Level of Consciousness (R41.82),impression_Burn - First degree (L55.0),impression_GI/GU - GERD (Reflux) (K21),impression_Injury - Face (S09.93),impression_Maltreatment - Child Sexual Abuse/Rape Suspected (T74.22),impression_Infectious - RSV (B97.4),impression_GI/GU - Constipation (K59.00),impression_Pain - Chest (presumed non-cardiac) (R07.89),impression_Injury - Eye and/or Orbit (S05),impression_Bleeding or Hematoma from Procedure/Medical Device (L76.22),impression_Sickle Cell Anemia/Crisis (D57.0),impression_EENT - Foreign Body to Eye (T15),impression_Newborn Care - Meconium Aspiration (P24.0),impression_Behavioral - Violent (R45.6),"impression_Injury - Wrist, Hand, or Fingers (S69.9)",impression_Maltreatment - Child Physical Abuse Suspected (T74.12),impression_Pain - Neck (Non-traumatic) (M54.2),impression_Injury - Foot (S99.92),impression_GI/GU - Abdominal Generalized (R10.84),impression_Respiratory - Hyperventilation (R06.4),impression_Neuro - Visual Disturbance (H53.9),impression_Endocrine - Hypoglycemia - Diabetic (E13.64),impression_GI/GU - Abdominal Pain/Problem (R10.84),impression_Infectious - Common Cold (J00),impression_CV - Cardiac Arrest (I46.9),impression_GI/GU - Pelvic or Perineal Pain (R10.2),impression_Mobility - Reduced (Z74.09),impression_Behavioral - Strange Behavior (R46.2),impression_CV - Hypovolemic Shock (T79.4XXA),impression_Environment - Altitude Sickness (T70.29),impression_Allergic Reaction (T78.40),impression_Injury - Nose (S09.92),impression_Respiratory - Pneumothorax (Spontaneous) (J93.9),"impression_CV - STEMI of Anterior Wall, With or Without Chest Pain (I21.0)",impression_Environment - Decompression Sickness (e.g. scuba diving) (T70.3),impression_OB - Contractions (O62.0),impression_Neuro - Status Epilepticus (G40.901),impression_Injury - Subdural Hemorrhage from Trauma (S06.5),impression_OB - Obstetric Trauma (O71.9),"impression_CV - STEMI of Inferior Wall, With or Without Chest Pain (I21.1)",impression_CV - Cardiac Arrest/Obvious Death (R99),impression_CV - Chest Pain - Angina (I20.0),impression_Paralysis - Quadriplegia (G82.50),impression_GI/GU - Hematemesis (vomiting blood) (K92.0),impression_GI/GU - Melena (bloody stools) (K92.1),impression_OB - Preterm Labor Without Delivery (O60.0),impression_Neuro - Altered Mental Status (R41.82),impression_Infectious - Croup (J05.0),impression_Injury - Lower leg (S89.9),impression_GI/GU - Bowel Obstruction (K56.60),impression_OB - Vomiting Due to Pregnancy (O21.9),impression_Endocrine - Adrenocortical Insufficiency (E27.40),impression_CV - STEMI of Inferior WallWith or Without Chest Pain (I21.1),impression_GI/GU - Nausea (Without Vomiting) (R11.0),impression_Intracranial - Stroke (CVA) Hemorrhagic (I62.9)"""
features = features.split(',')
cols = list(df.columns)
fea = [x for x in cols if x in features]
x = df[fea]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=lr, n_features_to_select=25)
rfe.fit(x_train, y_train)
l = get_vars(rfe, fea)
lr_ini25 = LogisticRegression(max_iter=1000)
x = df[l]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr_ini25.fit(x_train, y_train)
#print_performance(y_test, lr_15.predict(x_test))
ini25_df = create_coef_df(l, lr_ini25)
print(ini25_df)
print_performance(y_test, lr_ini25.predict(x_test))

                                             variable  coefficient        odd
0              impression_CV - Cardiac Arrest (I46.9)     4.069816  58.546170
1                           InitialIncidentType_SHOTF     2.513781  12.351539
2             impression_Behavioral - Violent (R45.6)     2.379839  10.803168
3   impression_Respiratory - Pulmonary Edema Acute...     2.359910  10.590000
4   impression_CV - STEMI of Anterior WallWith or ...     2.190727   8.941713
5                           InitialIncidentType_STABF     2.017841   7.522069
6   impression_Intracranial - Stroke (CVA) Hemorrh...     1.988483   7.304445
7   impression_CV - STEMI of other sitesWith or Wi...     1.955820   7.069712
8   impression_CV - Cardiac Arrest/Obvious Death (...     1.929846   6.888447
9             impression_Neuro - Unconscious (R40.20)     1.913761   6.778537
10     impression_Respiratory - Arrest/Apnea (R06.81)     1.710747   5.533093
11  impression_Injury - Lung Hemothorax - Traumati...     1.5824

In [24]:
# final stage 15 vars
col_name=list(df.columns)
col_name = [e for e in col_name if e not in ('Unnamed: 0','transloc_Level 1: 3 or More Providers (ALS Care)', 'transloc_Level 2: 2 Providers (ALS Care)',
                                    'transloc_Level 3: 1 Provider (ALS Care)',
                                    'transloc_Level 4: 1 Provider (BLS Care)',
                                    'transloc_No Transport LOC',
                                    'Count # of PatientID',
                                    'IncidentFirstDue',
                                    'is_SecondProvider',
                                    'is_SecondProvider2',
                                            'PrimaryKey')]
x = df[col_name]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=lr, n_features_to_select=15)
rfe.fit(x_train, y_train)
l = get_vars(rfe, col_name)
lr_final15 = LogisticRegression(max_iter=1000)
x = df[l]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr_final15.fit(x_train, y_train)
final15_df = create_coef_df(l, lr_final15)
print(final15_df)
print_performance(y_test, lr_final15.predict(x_test))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

                                             variable  coefficient        odd
0   procedure_Behavior - Restraint Applied (Physical)     2.763008  15.847442
1                    medication_Tranexamic Acid (TXA)     2.700356  14.885031
2   impression_CV - STEMI of Inferior Wall, With o...     2.622128  13.764986
3        procedure_Resp - Assisted Ventilation - CPAP     2.546710  12.765041
4                                      unit_type_SHRU     2.532702  12.587478
5                          hospital_name_Landing Zone     2.514150  12.356100
6   impression_CV - STEMI of Anterior Wall, With o...     2.471713  11.842720
7             impression_Behavioral - Violent (R45.6)     2.326209  10.239057
8   procedure_Resp - Assist Ventilation - BVM Via ...     2.287929   9.854504
9                           InitialIncidentType_SHOTF     2.259475   9.578063
10                            FinalIncidentType_STABF     2.245750   9.447503
11  impression_CV - STEMI of other sites, With or ...     2.1891

In [14]:
# final stage 25 vars
col_name=list(df.columns)
col_name = [e for e in col_name if e not in ('Unnamed: 0','transloc_Level 1: 3 or More Providers (ALS Care)', 'transloc_Level 2: 2 Providers (ALS Care)',
                                    'transloc_Level 3: 1 Provider (ALS Care)',
                                    'transloc_Level 4: 1 Provider (BLS Care)',
                                    'transloc_No Transport LOC',
                                    'Count # of PatientID',
                                    'IncidentFirstDue',
                                    'is_SecondProvider',
                                    'is_SecondProvider2',
                                            'PrimaryKey')]
x = df[col_name]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=lr, n_features_to_select=25)
rfe.fit(x_train, y_train)
l = get_vars(rfe, col_name)
lr_final25 = LogisticRegression(max_iter=1000)
x = df[l]
y = df['is_SecondProvider2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 112)
lr_final25.fit(x_train, y_train)
final25_df = create_coef_df(l, lr_final25)
print(final25_df)
print_performance(y_test, lr_final25.predict(x_test))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

                                             variable  coefficient        odd
0        procedure_Resp - Assisted Ventilation - CPAP     2.736316  15.430037
1   procedure_Behavior - Restraint Applied (Physical)     2.546789  12.766044
2                                      unit_type_SHRU     2.479569  11.936117
3              impression_CV - Cardiac Arrest (I46.9)     2.266309   9.643741
4                    medication_Tranexamic Acid (TXA)     2.218769   9.196003
5             impression_Behavioral - Violent (R45.6)     2.122257   8.349965
6            procedure_IO Start - Intraosseous Access     2.013504   7.489515
7                          hospital_name_Landing Zone     1.917319   6.802699
8   impression_CV - STEMI of Inferior Wall, With o...     1.853329   6.381027
9                             FinalIncidentType_STABF     1.842775   6.314038
10  impression_CV - STEMI of Anterior Wall, With o...     1.783140   5.948505
11  procedure_Resp - Assist Ventilation - BVM Via ...     1.7211

In [26]:
# save selected features
ini15_f = ['impression_CV - Cardiac Arrest (I46.9)',
 'InitialIncidentType_SHOTF',
 'impression_Respiratory - Pulmonary Edema Acute Onset (J81.0)',
 'impression_Behavioral - Violent (R45.6)',
 'impression_CV - STEMI of Anterior WallWith or Without Chest Pain (I21.0)',
 'InitialIncidentType_STABF',
 'impression_CV - Cardiac Arrest/Obvious Death (R99)',
 'impression_CV - STEMI of other sitesWith or Without Chest Pain (I21.2)',
 'impression_Intracranial - Stroke (CVA) Hemorrhagic (I62.9)',
 'impression_Neuro - Unconscious (R40.20)',
 'impression_Injury - Lung Hemothorax - Traumatic (S27.1)',
 'impression_Injury - Intracranial Injury Not Otherwise Specified (S06.9)',
 'impression_Respiratory - Arrest/Apnea (R06.81)',
 'impression_Vertigo (R42)',
 'impression_Neuro - Headache - Migraine (G43.9)']
ini25f = ['impression_CV - Cardiac Arrest (I46.9)',
 'InitialIncidentType_SHOTF',
 'impression_Behavioral - Violent (R45.6)',
 'impression_Respiratory - Pulmonary Edema Acute Onset (J81.0)',
 'impression_CV - STEMI of Anterior WallWith or Without Chest Pain (I21.0)',
 'InitialIncidentType_STABF',
 'impression_Intracranial - Stroke (CVA) Hemorrhagic (I62.9)',
 'impression_CV - STEMI of other sitesWith or Without Chest Pain (I21.2)',
 'impression_CV - Cardiac Arrest/Obvious Death (R99)',
 'impression_Neuro - Unconscious (R40.20)',
 'impression_Respiratory - Arrest/Apnea (R06.81)',
 'impression_Injury - Lung Hemothorax - Traumatic (S27.1)',
 'impression_Injury - Intracranial Injury Not Otherwise Specified (S06.9)',
 'impression_Neuro - Headache - Migraine (G43.9)',
 'impression_Vertigo (R42)',
 'impression_CV - Myocardial Infarction (Non-STEMI)With or Without Chest Pain (I21.4)',
 'impression_OB - Contractions (O62.0)',
 'impression_GI/GU - GU Problem Not Otherwise Listed (N39.9)',
 'impression_Injury - Lung Pneumothorax - Traumatic (S27.0)',
 'impression_Injury - Head with L.O.C. (S06.0X9A)',
 'impression_Pain - Chronic (G89.2)',
 'impression_OB - Preterm Labor with Delivery (O60.1)',
 'impression_Burn - Third degree (L55.2)',
 'impression_CV - STEMI of Inferior WallWith or Without Chest Pain (I21.1)',
 'impression_Infectious - Common Cold (J00)']
final25_f = ['procedure_Resp - Assisted Ventilation - CPAP',
 'procedure_Behavior - Restraint Applied (Physical)',
 'unit_type_SHRU',
 'impression_CV - Cardiac Arrest (I46.9)',
 'medication_Tranexamic Acid (TXA)',
 'impression_Behavioral - Violent (R45.6)',
 'procedure_IO Start - Intraosseous Access',
 'hospital_name_Landing Zone',
 'impression_CV - STEMI of Inferior Wall, With or Without Chest Pain (I21.1)',
 'FinalIncidentType_STABF',
 'impression_CV - STEMI of Anterior Wall, With or Without Chest Pain (I21.0)',
 'procedure_Resp - Assist Ventilation - BVM Via Mask',
 'InitialIncidentType_SHOTF',
 'medication_Midazolam (Versed)',
 'impression_CV - STEMI of Anterior WallWith or Without Chest Pain (I21.0)',
 'impression_Encounter for general examination without complaint, suspected or reported (Z00)',
 'impression_Neuro - Unconscious (R40.20)',
 'procedure_Laryngoscopy - Direct',
 'medication_Atropine (Atropine Sulfate) ',
 'impression_CV - STEMI of other sites, With or Without Chest Pain (I21.2)',
 'procedure_Airway - Clear/Suction',
 'impression_OB - Contractions (O62.0)',
 'impression_CV - Myocardial Infarction (Non-STEMI), With or Without Chest Pain (I21.4)',
 'reason_Specialty Center (Trauma STEMI Stroke)',
 'impression_Neuro - Headache - Migraine (G43.9)']
final15_f = ['procedure_Behavior - Restraint Applied (Physical)',
 'medication_Tranexamic Acid (TXA)',
 'impression_CV - STEMI of Inferior Wall, With or Without Chest Pain (I21.1)',
 'procedure_Resp - Assisted Ventilation - CPAP',
 'unit_type_SHRU',
 'hospital_name_Landing Zone',
 'impression_CV - STEMI of Anterior Wall, With or Without Chest Pain (I21.0)',
 'impression_Behavioral - Violent (R45.6)',
 'procedure_Resp - Assist Ventilation - BVM Via Mask',
 'InitialIncidentType_SHOTF',
 'FinalIncidentType_STABF',
 'impression_CV - STEMI of other sites, With or Without Chest Pain (I21.2)',
 'impression_CV - Cardiac Arrest (I46.9)',
 'procedure_IO Start - Intraosseous Access',
 'impression_CV - STEMI of Anterior WallWith or Without Chest Pain (I21.0)']